In [ ]:
!pip install sentence-transformers
!pip install torch
!pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import json
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Download the punkt_tab data
from nltk.tokenize import sent_tokenize

embedder = SentenceTransformer('all-MiniLM-L6-v2')
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2", framework="pt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
Device set to use cpu


In [ ]:
# Load movie plots
with open("/content/plots.json", "r") as f:
    plots = json.load(f)

In [ ]:
paragraphs = []
metadata = []  # Keep track of which paragraph belongs to which movie

for plot in plots:
    sentences = sent_tokenize(plot['text'])
    chunk_size = 4  # 4 sentences per paragraph (you can tune this)
    for i in range(0, len(sentences), chunk_size):
        paragraph = ' '.join(sentences[i:i+chunk_size])
        paragraphs.append(paragraph)
        metadata.append(plot['title'])

# Embed all paragraphs
paragraph_embeddings = embedder.encode(paragraphs, convert_to_tensor=True)

In [ ]:
def find_best_paragraph(query):
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, paragraph_embeddings)[0]
    best_idx = torch.argmax(cos_scores).item()
    return metadata[best_idx], paragraphs[best_idx]

def answer_question(query):
    movie_title, best_paragraph = find_best_paragraph(query)
    answer = qa_pipeline(question=query, context=best_paragraph)
    return movie_title, answer['answer']


In [ ]:
question = "What happens to Cooper in the end?"
movie, answer = answer_question(question)
print(f"Movie: {movie}\nAnswer: {answer}")

Movie: Interstellar
Answer: He reunites with Murph


In [ ]:
question = "How does Andy escape from prison?"
movie, answer = answer_question(question)
print(f"Movie: {movie}\nAnswer: {answer}")

Movie: The Shawshank Redemption
Answer: he dreams of living in Zihuatanejo


In [ ]:
question = "What is the moon in Moonfall?"
movie, answer = answer_question(question)
print(f"Movie: {movie}\nAnswer: {answer}")

Movie: Moonfall
Answer: hollow
